In [ ]:
!pip install smolagents polars pydantic
!pip install --upgrade smolagents litellm polars pydantic


In [ ]:
from smolagents import CodeAgent
from smolagents.models import LiteLLMModel
import polars as pl
from pydantic import BaseModel
import random

# -----------------------------
# 1️⃣ Data Model
# -----------------------------
class Question(BaseModel):
    question: str
    answer: str

# -----------------------------
# 2️⃣ Question Bank
# (Add more questions anytime)
# -----------------------------
QUESTIONS = [
    Question(question="What is 3 + 3?", answer="6"),
    Question(question="What color is the sun?", answer="yellow"),
    Question(question="How many legs does a cat have?", answer="4"),
    Question(question="What is 10 - 4?", answer="6"),
    Question(question="What comes after D?", answer="e"),
    Question(question="How many days in a week?", answer="7"),
    Question(question="What color are bananas?", answer="yellow"),
    Question(question="What is 2 x 5?", answer="10"),
    Question(question="How many wheels does a car have?", answer="4"),
    Question(question="What is the first letter of alphabet?", answer="a"),
]

# -----------------------------
# 3️⃣ Score Table
# -----------------------------
score = pl.DataFrame({"correct": [0], "wrong": [0]})

# -----------------------------
# 4️⃣ LLM Model (required by smolagents)
# -----------------------------
model = LiteLLMModel(
    model="gpt-3.5-turbo",
    model_id="gpt-3.5-turbo"
)

# -----------------------------
# 5️⃣ Agent (not used directly, but valid)
# -----------------------------
quiz_bot = CodeAgent(
    name="QuizWizard",
    tools=[],
    model=model
)

# -----------------------------
# 6️⃣ Game Logic
# -----------------------------
def play_game():
    global score
    print("\n🎮 Welcome to QUIZ WIZARD 🎮\n")

    num_questions = min(10, len(QUESTIONS))
    selected_questions = random.sample(QUESTIONS, num_questions)

    for q in selected_questions:
        print(f"🤖 {q.question}")
        answer = input("🧒 Your answer: ").strip().lower()

        if answer == q.answer.lower():
            print("🎉 Correct!\n")
            score = score.with_columns(
                (pl.col("correct") + 1).alias("correct")
            )
        else:
            print(f"😅 Wrong! Answer: {q.answer}\n")
            score = score.with_columns(
                (pl.col("wrong") + 1).alias("wrong")
            )

    show_result()

# -----------------------------
# 7️⃣ Result
# -----------------------------
def show_result():
    c, w = score.row(0)
    print("\n🏆 GAME OVER 🏆")
    print(f"✅ Correct: {c}")
    print(f"❌ Wrong: {w}")

    if c >= 7:
        print("🤖 AMAZING! You are a quiz master 🧠✨")
    else:
        print("🤖 Keep practicing! You’re doing great 🚀")

# -----------------------------
# 8️⃣ Start
# -----------------------------
if __name__ == "__main__":
    play_game()


In [ ]:
# !pip install smolagents litellm polars pydantic

import os
os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY" # <-- REPLACE WITH YOUR ACTUAL GOOGLE API KEY

from smolagents import CodeAgent
from smolagents.models import LiteLLMModel
from pydantic import BaseModel
import polars as pl
import json

# -----------------------------
# 1️⃣ Question Schema
# -----------------------------
class Question(BaseModel):
    question: str
    answer: str

# -----------------------------
# 2️⃣ Score Table
# -----------------------------
score = pl.DataFrame({"correct": [0], "wrong": [0]})

# -----------------------------
# 3️⃣ LLM Model
# -----------------------------
model = LiteLLMModel(
    model="gemini-pro" # Changed model to gemini-pro
)


# -----------------------------
# 4️⃣ Agent
# -----------------------------
quiz_agent = CodeAgent(
    name="AgeQuizWizard",
    tools=[],
    model=model
)

# -----------------------------
# 5️⃣ Age Group Selection
# -----------------------------
def select_age_group():
    print("""
Choose Age Group:
1️⃣ 1-5 years
2️⃣ 6-10 years
3️⃣ 11-15 years
4️⃣ 16-20 years
5️⃣ 21-25 years
6️⃣ 26-30 years
""")
    choice = input("Enter choice (1-6): ").strip()

    age_map = {
        "1": "very easy (colors, animals, numbers)",
        "2": "easy (basic math, objects, nature)",
        "3": "medium (school-level science, math)",
        "4": "hard (logic, reasoning, academics)",
        "5": "advanced (general knowledge, logic)",
        "6": "advanced adult-level knowledge"
    }

    return age_map.get(choice, "easy")

# -----------------------------
# 6️⃣ Generate Questions via LLM
# -----------------------------
def generate_questions(difficulty):
    prompt = f"""
You are a quiz generator. Generate age-appropriate questions for {difficulty} learners. Generate exactly 10 quiz questions.
Return ONLY JSON in this format:

[
  {{ "question": "...", "answer": "..." }}
]
"""
    response = quiz_agent.run(prompt)

    questions_data = json.loads(response)
    return [Question(**q) for q in questions_data]

# -----------------------------
# 7️⃣ Quiz Logic
# -----------------------------
def play_game(questions):
    global score
    print("\n🎮 QUIZ STARTED 🎮\n")

    for q in questions:
        print(f"🤖 {q.question}")
        user_answer = input("🧒 Your answer: ").strip().lower()

        if user_answer == q.answer.lower():
            print("🎉 Correct!\n")
            score = score.with_columns(
                (pl.col("correct") + 1).alias("correct")
            )
        else:
            print(f"😅 Wrong! Correct answer: {q.answer}\n")
            score = score.with_columns(
                (pl.col("wrong") + 1).alias("wrong")
            )

    show_result()

# -----------------------------
# 8️⃣ Result
# -----------------------------
def show_result():
    c, w = score.row(0)
    print("\n🏆 QUIZ COMPLETED 🏆")
    print(f"✅ Correct: {c}/10")
    print(f"❌ Wrong: {w}/10")

    if c >= 8:
        print("🌟 EXCELLENT! You are brilliant 🧠")
    elif c >= 5:
        print("👍 GOOD JOB! Keep learning 📘")
    else:
        print("💪 Keep practicing! You’ll improve 🚀")

# -----------------------------
# 9️⃣ Start
# -----------------------------
if __name__ == "__main__":
    difficulty = select_age_group()
    questions = generate_questions(difficulty)
    play_game(questions)

In [ ]:
# Hybride

import random
import polars as pl
from pydantic import BaseModel

# -----------------------------
# Question Model
# -----------------------------
class Question(BaseModel):
    question: str
    answer: str

# -----------------------------
# Offline Question Bank
# -----------------------------
QUESTION_BANK = {
    "1-5": [
        Question(question="What color is an apple?", answer="red"),
        Question(question="How many eyes do you have?", answer="2"),
        Question(question="What sound does a dog make?", answer="bark"),
        Question(question="What color is the sky?", answer="blue"),
        Question(question="How many hands do you have?", answer="2"),
    ],
    "6-10": [
        Question(question="What is 5 + 5?", answer="10"),
        Question(question="How many days in a week?", answer="7"),
        Question(question="Which planet do we live on?", answer="earth"),
        Question(question="What is 10 - 3?", answer="7"),
        Question(question="What comes after C?", answer="d"),
    ],
    "11-15": [
        Question(question="What is 12 x 2?", answer="24"),
        Question(question="What gas do plants breathe in?", answer="carbon dioxide"),
        Question(question="Who discovered gravity?", answer="newton"),
        Question(question="What is 100 / 4?", answer="25"),
        Question(question="What is H2O?", answer="water"),
    ],
    "16-30": [
        Question(question="What is the capital of India?", answer="new delhi"),
        Question(question="What is Python?", answer="programming language"),
        Question(question="What does CPU stand for?", answer="central processing unit"),
        Question(question="What is 2**3?", answer="8"),
        Question(question="Who invented the internet?", answer="arpa"),
    ]
}

# -----------------------------
# Score
# -----------------------------
score = pl.DataFrame({"correct": [0], "wrong": [0]})

# -----------------------------
# Age Selection
# -----------------------------
def select_age_group():
    print("""
Choose Age Group:
1️⃣ 1-5 years
2️⃣ 6-10 years
3️⃣ 11-15 years
4️⃣ 16-30 years
""")
    choice = input("Enter choice: ").strip()

    return {
        "1": "1-5",
        "2": "6-10",
        "3": "11-15",
        "4": "16-30"
    }.get(choice, "6-10")

# -----------------------------
# Quiz Logic
# -----------------------------
def play_game(age_group):
    global score

    questions = QUESTION_BANK[age_group]
    selected = random.sample(questions, min(10, len(questions)))

    print("\n🎮 QUIZ STARTED 🎮\n")

    for q in selected:
        print(f"🤖 {q.question}")
        answer = input("🧒 Your answer: ").strip().lower()

        if answer == q.answer.lower():
            print("🎉 Correct!\n")
            score = score.with_columns((pl.col("correct") + 1).alias("correct"))
        else:
            print(f"😅 Wrong! Answer: {q.answer}\n")
            score = score.with_columns((pl.col("wrong") + 1).alias("wrong"))

    show_result()

# -----------------------------
# Result
# -----------------------------
def show_result():
    c, w = score.row(0)
    print("\n🏆 QUIZ OVER 🏆")
    print(f"✅ Correct: {c}")
    print(f"❌ Wrong: {w}")

    if c >= 7:
        print("🌟 Excellent work!")
    else:
        print("💪 Keep practicing!")

# -----------------------------
# Start
# -----------------------------
if __name__ == "__main__":
    age = select_age_group()
    play_game(age)
